In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from scipy.fft import dct
import math

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Resize((32, 32)),
     transforms.Normalize((0.5,), (0.5,))])

batch_size = 4

trainset = torchvision.datasets.MNIST(root='../data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='../data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


D:\TU_KL\thesis\cnn_comp\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
class DCT_layer(nn.Module):
    def __init__(self,out_features: int):
        super(DCT_layer, self).__init__()
        
        self.out_features = out_features
#         self.in_features = in_features
        
        default_dtype = torch.get_default_dtype()
#         self.register_parameter('y', torch.nn.Parameter(torch.zeros(self.out_features, dtype=default_dtype)))
        self.fc = nn.Parameter(torch.empty((self.out_features,batch_size), dtype=default_dtype))
        
        self.reset_parameters()
        
        self.mat = nn.Parameter(self.dct_mat(batch_size))
#         self.y.register_hook(lambda grad: grad)
#         self.fc.register_hook(lambda grad: grad)
        
        
        
    def dct2(self, x, norm=None):
        N = len(x)
        n = np.arange(N)

        coeff_vec = []

        for k in n:
            yk = 0
            for i in n:
                yk += x[i] * np.cos(np.pi* k*(2*i + 1)/(2*N))
            if norm=='ortho':
                if k == 0:
                    yk = np.sqrt(1/(4*N)) * yk
                else:
                    yk = np.sqrt(1/(2*N)) * yk
            yk = yk*2
            coeff_vec.append(yk)
        
#         print(torch.stack(coeff_vec))
        return np.array(coeff_vec)

#     def dct_mat(self,x):  ## generate dct matrix
#         dct_mat = dct(np.eye(x.shape[-1]),norm='ortho')
#         return torch.from_numpy(dct_mat).float()

    
    def dct_mat(self,batch_size): ## orthogonal dct matrix
        N = batch_size
        N = torch.tensor(N)
        
        default_dtype = torch.get_default_dtype()
        
        C = torch.zeros(N,N, dtype=default_dtype)
        k = n = torch.arange(N)
        
        C[0,n] = torch.sqrt(1/N)           
        for _k in k[1:]:
            C[_k,n] = (torch.sqrt(2/N)*torch.cos((np.pi*_k*(1/2+n))/N)).float()
        
#         return torch.from_numpy(C).float().to(device)
        return C

    def dct_comp(self,x):
        
#         dct_x = self.dct2(x.detach().numpy(),norm='ortho')
#         return torch.tensor(dct_x)
        return self.mat.T @ x ## self.mat.T to make it same as scipy dct matrix
    
    def reset_parameters(self) -> None:
        nn.init.kaiming_uniform_(self.fc, a=math.sqrt(5), nonlinearity='relu')
        
    def forward(self,x):
#         self.dummy_input = nn.Parameter(torch.empty(1,x.shape[-1]))
#         nn.init.kaiming_uniform_(self.dummy_input, a=math.sqrt(5), nonlinearity='relu')
        
        dct_x = nn.Parameter(self.dct_comp(x)) 
#         print(dummy_input.shape)
#         print(self.fc.shape)
#         print(dct_x.shape)
        
#         dct_x = torch.reshape(dct_x,(x.shape[-1],1)).float()
        
#         print(dct_x.shape)
#         print(dct_x_rs.dtype)
#         print(self.fc.dtype)
        
        w = nn.Parameter(self.fc @ dct_x) ##
        
#         w = w.T
#         print(w.shape)
        y = F.linear(x,w)
        
#         print(y.shape)
        return y

In [4]:
class DCT_LeNet(nn.Module):

    def __init__(self):
        super(DCT_LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 120, 5)  
        
#         self.fc1 = nn.Linear(120, 84)
        self.dct = DCT_layer(84)
        self.fc2 = nn.Linear(84, 10)


    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = F.relu(self.conv3(x))
        
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
#         x = F.relu(self.dct(x))
#         x = self.fc2(x)
        x = F.relu(self.dct(x))
        x = self.fc2(x) 
        return x


dct_net = DCT_LeNet()
print(dct_net)


DCT_LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (dct): DCT_layer()
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
class LeNet(nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 120, 5)  
        
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = F.relu(self.conv3(x))
        
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


lenet = LeNet()
print(lenet)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)


In [6]:
def train(dataloader,model,criterion,optimizer):

    train_loss = 0.0
    for X, y in dataloader:
        inputs, labels = X, y
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        train_loss += loss.item()*inputs.size(0)
    train_loss = train_loss/len(dataloader)
    
    print(f'Training Loss: {train_loss:.8f}')

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X, y
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(dct_net.parameters(), lr=0.001, momentum=0.9)

print("DCT_LeNet")
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, dct_net, criterion, optimizer)
    test(testloader, dct_net, criterion)


DCT_LeNet
Epoch 1
-------------------------------


D:\TU_KL\thesis\cnn_comp\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Training Loss: 6.78275626
Test Error: 
 Accuracy: 38.6%, Avg loss: 1.449770 

Epoch 2
-------------------------------
Training Loss: 5.68868047
Test Error: 
 Accuracy: 47.2%, Avg loss: 1.333088 

Epoch 3
-------------------------------
Training Loss: 5.21988878
Test Error: 
 Accuracy: 47.6%, Avg loss: 1.286996 



In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lenet.parameters(), lr=0.001, momentum=0.9)

print("LeNet")
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, lenet, criterion, optimizer)
    test(testloader, lenet, criterion)


LeNet
Epoch 1
-------------------------------
Training Loss: 0.97524523
Test Error: 
 Accuracy: 98.1%, Avg loss: 0.061370 

Epoch 2
-------------------------------
Training Loss: 0.23062377
Test Error: 
 Accuracy: 98.8%, Avg loss: 0.039129 

Epoch 3
-------------------------------
Training Loss: 0.16029771
Test Error: 
 Accuracy: 99.0%, Avg loss: 0.035459 

